In [47]:
import numpy as np
import pandas as pd

In [48]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [49]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [50]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [51]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [129]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [54]:
model = Sequential([
    Dense(32, activation = 'relu',input_dim = 8),
    Dense(1,activation = 'sigmoid')
])
model.compile(optimizer = 'Adam', loss ='binary_crossentropy',metrics = ['accuracy'])

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [55]:
model.fit(X_train,y_train,batch_size = 32, epochs = 100,validation_data = (X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5717 - loss: 0.7252 - val_accuracy: 0.6234 - val_loss: 0.6853
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6189 - loss: 0.6779 - val_accuracy: 0.6623 - val_loss: 0.6455
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6417 - loss: 0.6405 - val_accuracy: 0.6948 - val_loss: 0.6125
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6792 - loss: 0.6094 - val_accuracy: 0.7143 - val_loss: 0.5860
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6971 - loss: 0.5828 - val_accuracy: 0.7468 - val_loss: 0.5629
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7248 - loss: 0.5603 - val_accuracy: 0.7727 - val_loss: 0.5453
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7410 - loss: 0.5415 - val_accuracy: 0.7987 - val_loss: 0.5298
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7557 - loss: 0.5261 - val_accuracy: 0.7987 - 

### 1. How to select appropriate optimizer ?
### 2. Number of Nodes in a layer ?
### 3. Number of Hidden layers ?
### 4. All in one model

## 1. How to select appropriate optimizer ?

In [82]:
import keras_tuner as kt

In [85]:
def build_model1(hp):
    model = Sequential([
        Dense(32, activation = 'relu',input_dim = 8),
        Dense(1,activation = 'sigmoid')
    ])
    optimizer = hp.Choice('optimizer',values = ['adam','sgd','adadelta','rmsprop'])
    model.compile(optimizer = optimizer, loss ='binary_crossentropy',metrics = ['accuracy'])
    return model

In [87]:
tuner1 = kt.RandomSearch(build_model1,objective = 'val_accuracy',max_trials = 5)

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [88]:
tuner1.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 12s


In [89]:
tuner1.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [90]:
model1 = tuner1.get_best_models(num_models = 1)[0]

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [91]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [92]:
model1.fit(X_train,y_train,batch_size=32,epochs = 100,initial_epoch=6,validation_data = (X_test,y_test))

Epoch 7/100


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7573 - loss: 0.5129 - val_accuracy: 0.7727 - val_loss: 0.5116
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7622 - loss: 0.4964 - val_accuracy: 0.7727 - val_loss: 0.5057
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7671 - loss: 0.4863 - val_accuracy: 0.7857 - val_loss: 0.5001
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7720 - loss: 0.4789 - val_accuracy: 0.7857 - val_loss: 0.4976
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7785 - loss: 0.4734 - val_accuracy: 0.7857 - val_loss: 0.4961
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7785 - loss: 0.4687 - val_accuracy: 0.7662 - val_loss: 0.4971
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7801 - loss: 0.4647 - val_accuracy: 0.7662 - val_loss: 0.4976
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7769 - loss: 0.4615 - val_accuracy: 0.7662 - val_los

## 2. Number of Nodes in a layer ?

In [93]:
def build_model2(hp):
    units = hp.Int('units',min_value = 8,max_value = 128,step = 8)
    model = Sequential([
        Dense(units = units, activation = 'relu',input_dim = 8),
        Dense(1,activation = 'sigmoid')
    ])
    model.compile(optimizer = 'rmsprop', loss ='binary_crossentropy',metrics = ['accuracy'])
    return model

In [94]:
tuner2 = kt.RandomSearch(build_model2, objective = 'val_accuracy',max_trials = 5,directory = 'my_dir')

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [95]:
tuner2.search(X_train,y_train,epochs = 5,validation_data =(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 15s


In [96]:
tuner2.get_best_hyperparameters()[0].values

{'units': 104}

In [97]:
model2 = tuner2.get_best_models(num_models = 1)[0]

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [98]:
model2.fit(X_train,y_train,batch_size = 32,epochs =100,initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7752 - loss: 0.4684
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7769 - loss: 0.4573 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7834 - loss: 0.4523
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7818 - loss: 0.4476
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7883 - loss: 0.4441 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7866 - loss: 0.4412
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7801 - loss: 0.4380
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7850 - loss: 0.4363 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7818 - loss: 0.4346 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7850 - loss: 0.4329 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7834 - loss: 0.4316 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

## Number of hidden layers?

In [107]:
def built_models3(hp):
    model = Sequential()
    model.add(Dense(72,activation = 'relu',input_dim = 8))
    for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):
        model.add(Dense(72,activation = 'relu'))
    
    model.add(Dense(1,activation= 'sigmoid'))
    model.compile(optimizer='rmsprop',loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model

In [108]:
tuner3 = kt.RandomSearch(built_models3,objective='val_accuracy',max_trials = 3,directory = 'my_dir',project_name = 'num_layers')

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [109]:
tuner3.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 14s


In [110]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [111]:
model3 = tuner3.get_best_models(num_models = 1)[0]

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [112]:
model3.fit(X_train,y_train,batch_size = 32,epochs =100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7883 - loss: 0.4411 - val_accuracy: 0.7662 - val_loss: 0.5350
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7850 - loss: 0.4250 - val_accuracy: 0.7597 - val_loss: 0.5429
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7850 - loss: 0.4207 - val_accuracy: 0.7532 - val_loss: 0.5381
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7948 - loss: 0.4144 - val_accuracy: 0.7338 - val_loss: 0.5449
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8062 - loss: 0.4076 - val_accuracy: 0.7532 - val_loss: 0.5372
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8029 - loss: 0.4041 - val_accuracy: 0.7403 - val_loss: 0.5460
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8111 - loss: 0.3963 - val_accuracy: 0.7532 - val_loss: 0.5603
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8160 - loss: 0.3960 - val_accuracy: 0.7468 - val_lo

In [146]:
def built_models_final(hp):
    model = Sequential()

    counter = 0


    for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i),min_value = 8,max_value = 128,step = 8),
                    activation = hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid']),
                    input_dim = 8
            )) 
            # model.add(Dropout(
                # hp.Choice('dropout' + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
            # ))
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i),min_value = 8,max_value = 128,step = 8),
                    activation = hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid'])
            ))
            # model.add(Dropout(
            #     hp.Choice('dropout' + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
            # ))
        counter += 1
    model.add(Dense(1,activation= 'sigmoid'))
    model.compile(
        optimizer=hp.Choice('optimizer',values = ['adam','rmsprop','sgd','adadelta','nadam']),
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
        )
    return model

In [147]:
tuner_final = kt.RandomSearch(built_models_final,objective = 'val_accuracy',max_trials = 3,directory = 'my_dir',project_name = 'final_tuning')

In [150]:
tuner_final.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [151]:
tuner_final.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 16,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'sigmoid',
 'units2': 48,
 'activation2': 'tanh',
 'units3': 32,
 'activation3': 'tanh',
 'units4': 24,
 'activation4': 'sigmoid'}

In [152]:
final_model = tuner_final.get_best_models(num_models = 1)[0]

/home/om-codex/ml/ml_env/lib64/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [153]:
final_model.fit(X_train,y_train,epochs = 200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7573 - loss: 0.4909 - val_accuracy: 0.7078 - val_loss: 0.5263
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7524 - loss: 0.4831 - val_accuracy: 0.7273 - val_loss: 0.5299
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7606 - loss: 0.4743 - val_accuracy: 0.7468 - val_loss: 0.5156
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7557 - loss: 0.4692 - val_accuracy: 0.7273 - val_loss: 0.5175
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7622 - loss: 0.4688 - val_accuracy: 0.7208 - val_loss: 0.5168
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7655 - loss: 0.4644 - val_accuracy: 0.7403 - val_loss: 0.5109
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7736 - loss: 0.4608 - val_accuracy: 0.7338 - val_loss: 0.5147
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7720 - loss: 0.4597 - val_accuracy: 0.772